In [1]:
import os
import time
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [4]:
assistant = client.beta.assistants.create(
    name="쉬운 말 추천v1",
    instructions="문장에서 어려운 단어를 식별하고 더 쉬운 표현을 제안하세요.",
    model="gpt-4o-mini",
)

In [5]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="매니지드 서비스는 제품 설치부터 점검, 운영까지 대신해 준다."
)

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [14]:
assistant.dict

<bound method BaseModel.dict of Assistant(id='asst_d4Qk2GghBL1DzBEBBCxGzFFM', created_at=1738649276, description=None, instructions='문장에서 어려운 단어를 식별하고 더 쉬운 표현을 제안하세요.', metadata={}, model='gpt-4o-mini', name='쉬운 말 추천v1', object='assistant', tools=[], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0, reasoning_effort=None)>

In [13]:
thread.dict

<bound method BaseModel.dict of Thread(id='thread_O2Jm8gri70hvipftAZzTJuYR', created_at=1738649277, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))>

In [7]:
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    run_status = run.status

    if run_status in ["completed", "failed"]:
        break

    time.sleep(1)

if run_status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages.data[0].content[0].text.value)
else:
    print(f"Run status: {run_status}")

"매니지드 서비스는 제품 설치부터 점검, 운영까지 대신해 준다."에서 어려운 단어는 "매니지드 서비스"입니다. 

더 쉬운 표현으로 바꾸면: 

"관리 서비스는 제품 설치부터 점검, 운영까지 대신해 준다." 

라고 할 수 있습니다.


In [15]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_d4Qk2GghBL1DzBEBBCxGzFFM', deleted=True, object='assistant.deleted')

In [10]:
import requests
import json

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [13]:
def get_simpler_alternatives(keyword):
    url = f"https://plainkorean.kr/api.jsp?keyword={keyword}"
    print(f"Requesting: {url}")
    response = requests.get(url, verify=False)
    print(f"Response status code: {response.status_code}")
    if response.status_code == 200:
        result = response.json()
        print(f"Response content: {response}")
        if result:
            return json.dumps(result[0], ensure_ascii=False)
    return json.dumps({"error": "No alternatives found"})

In [14]:
get_simpler_alternatives_json = {
    "name": "get_simpler_alternatives",
    "description": "Get simpler alternatives for a given Korean keyword from plainkorean.kr API",
    "parameters": {
        "type": "object",
        "properties": {
            "keyword": {
                "type": "string",
                "description": "The Korean keyword to find simpler alternatives for",
            },
        },
        "required": ["keyword"],
    },
}

In [15]:
assistant = client.beta.assistants.create(
    name="쉬운 말 추천v2",
    instructions="문장에서 어려운 단어를 식별하고, 제공된 함수를 사용해 더 쉬운 표현을 제안하세요.",
    model="gpt-4o-mini",
    tools=[{"type": "function", "function": get_simpler_alternatives_json}],
)

In [16]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="매니지드 서비스는 제품 설치부터 점검, 운영까지 대신해 준다.",
)

In [17]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [18]:
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    run_status = run.status

    if run_status == "requires_action" and run.required_action is not None:
        tools_to_call = run.required_action.submit_tool_outputs.tool_calls
        tool_output_array = []
        for tool in tools_to_call:
            tool_call_id = tool.id
            function_name = tool.function.name
            function_arg = json.loads(tool.function.arguments)
            if function_name == 'get_simpler_alternatives':
                output = get_simpler_alternatives(function_arg["keyword"])
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_output_array,
        )
    elif run_status in ["completed", "failed"]:
        break

    time.sleep(1)

Requesting: https://plainkorean.kr/api.jsp?keyword=매니지드
Response status code: 200
Response content: <Response [200]>
Requesting: https://plainkorean.kr/api.jsp?keyword=서비스
Response status code: 200
Response content: <Response [200]>
Requesting: https://plainkorean.kr/api.jsp?keyword=점검
Response status code: 200
Response content: <Response [200]>
Requesting: https://plainkorean.kr/api.jsp?keyword=운영
Response status code: 200
Response content: <Response [200]>


In [21]:
tool_output_array

[{'tool_call_id': 'call_iQV9xzBU2ooESKP7l1uUjWmZ',
  'output': '{"alt": "위탁 관리 서비스", "keyword": "매니지드 서비스", "example": ["(X) 클라우드의 성공은 \'매니지드 서비스\'가 좌우한다.", "(O) 자원 공유의 성공은 \'위탁 관리 서비스\'가 좌우한다.", "(X) 매니지드 서비스는 제품 설치부터 점검, 운영까지 대신해 준다.", "(O) 위탁 관리 서비스는 제품 설치부터 점검, 운영까지 대신해 준다.", "(X) 구독 서비스에서 한 단계 확장한 매니지드 서비스도 제공한다. ", "(O) 구독 서비스에서 한 단계 확장한 위탁 관리 서비스도 제공한다."]}'},
 {'tool_call_id': 'call_yTYMj6VRhKX0EGbiAROlPg17',
  'output': '{"alt": "빈말, 입발림", "keyword": "립 서비스", "example": ["(X) 립 서비스만 할 게 아니라 기후 행동에 동참해야 한다.", "(O) 반말만 할 게 아니라 기후 행동에 동참해야 한다.", "(X) 청년이 미래라고 립 서비스만 할 것이 아니라 실행에 나서야 한다.", "(O) 청년이 미래라고 입발림만 할 것이 아니라 실행에 나서야 한다.", "(X) 구조 조정이 없을 것이라는 회사의 발표는 립 서비스로 보인다.", "(O) 구조 조정이 없을 것이라는 회사의 발표는 빈말로 보인다."]}'},
 {'tool_call_id': 'call_2HldxgqnS5HoLElS74ASHYon',
  'output': '{"error": "No alternatives found"}'},
 {'tool_call_id': 'call_oxXGOJJlwu1E1CmFAIZImqDj',
  'output': '{"error": "No alternatives found"}'}]

In [19]:
if run_status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages.data[0].content[0].text.wvalue)
else:
    print(f"Run status: {run_status}")

문장에서의 어려운 단어들을 더 쉬운 표현으로 바꾸면 다음과 같습니다:

- "매니지드" → "위탁 관리 서비스"
- "서비스" → "믹서, 입회물" (대체 표현으로 사용되지 않는 경우)
- "점검" → (대체 표현 없음)
- "운영" → (대체 표현 없음)

따라서, 전체 문장을 더 쉬운 표현으로 바꾸면:

"위탁 관리 서비스는 제품 설치부터 점검, 운영까지 대신해 준다."

단어 "점검"과 "운영"에 대한 더 쉬운 표현은 없으므로 그대로 사용했습니다.


In [20]:
client.beta.assistants.delete(assistant.id)

AssistantDeleted(id='asst_JW5mPnmLoGCPcL1fh5XywpkS', deleted=True, object='assistant.deleted')

In [2]:
from tavily import TavilyClient

tavily_client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [12]:
assistant_instructions = """
You create a glossary entry in Korean on a given term.

Use the web_search tool for initial research to gather and verify information from credible sources. This ensures that definitions are informed by the most recent and reliable data.

If the tool does not return any information, abort with fail message.

Before including a URL, verify its validity and ensure it leads to the specific content being referenced. Avoid using generic homepage URLs unless they directly relate to the content. Never fabricate a fictional URL.

Instead of using honorifics (e.g. "입니다") in sentences, use haereahe (e.g. "이다") to maintain a direct and concise tone.

Follow output format below:
```
[Term]란 [comprehensive definition in 2-3 paragraphs].

### 참고

{% for each reference %}
- {%=reference in APA style. If the author and site name are not the same, write the author and site name separately.}
{% end for %}
```
"""

In [13]:
def web_search(query):
    # search_depth : basic(rapid response), advanced(focusing on quality)
    search_result = tavily_client.get_search_context(query, search_depth="advanced", max_tokens=8000)
    print(search_result)
    return search_result

In [14]:
web_search_json = {
    "name": "web_search",
    "description": "Get recent information from the web.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {"type": "string", "description": "The search query to use."},
        },
        "required": ["query"]
    }
}

In [15]:
assistant = client.beta.assistants.create(
    name="Define it!",
    instructions=assistant_instructions,
    model="gpt-4o-mini",
    tools=[{"type": "function", "function": web_search_json}],
)

In [16]:
thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Large Multimodal Models",
)

In [17]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

In [18]:
while True:
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id,
    )
    run_status = run.status

    if run_status == "requires_action" and run.required_action is not None:
        tools_to_call = run.required_action.submit_tool_outputs.tool_calls
        tool_output_array = []
        for tool in tools_to_call:
            tool_call_id = tool.id
            function_name = tool.function.name
            function_arg = json.loads(tool.function.arguments)
            if function_name == 'web_search':
                output = web_search(function_arg["query"])
            tool_output_array.append({"tool_call_id": tool_call_id, "output": output})

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_output_array,
        )
    elif run_status in ["completed", "failed"]:
        break

    time.sleep(1)

"[\"{\\\"url\\\": \\\"https://klu.ai/glossary/large-multimodal-models\\\", \\\"content\\\": \\\"Large Multimodal Models (LMMs), also known as Multimodal Large Language Models (MLLMs), are advanced AI systems that can process and generate information across multiple data modalities, such as text, images, audio, and video. Unlike traditional AI models that are typically limited to a single type of data, LMMs can understand and synthesize information from various sources, providing a more\\\"}\", \"{\\\"url\\\": \\\"https://www.shaip.com/blog/what-are-large-multimodal-models-lmms/\\\", \\\"content\\\": \\\"Large Multimodal Models (LMMs) are a revolution in artificial intelligence (AI). Unlike traditional AI models that operate within a single data environment such as text, images, or audio, LMMs are capable of creating and processing multiple modalities simultaneously. Hence the generation of outputs with context-aware multimedia information.\\\"}\", \"{\\\"url\\\": \\\"https://www.movewo

In [19]:
if run_status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
    )
    print(messages.data[0].content[0].text.value)
else:
    print(f"Run status: {run_status}")

```
Large Multimodal Models란 텍스트, 이미지, 오디오, 비디오 등 다양한 데이터 유형을 처리하고 생성할 수 있는 고급 인공지능 시스템이다. 이러한 모델은 단일 데이터 유형에만 국한되지 않고, 여러 소스에서 정보를 이해하고 합성할 수 있는 능력을 지닌다. 결과적으로, LMMs는 보다 풍부하고 맥락을 반영한 멀티미디어 정보를 생성할 수 있는 장점을 제공한다.

LMMs는 대용량 데이터셋에서 훈련되어 다양한 데이터 모달리티 간의 관계를 학습한다. 현재 가장 널리 사용되는 다중 모달 언어 모델은 텍스트와 이미지에 초점을 맞추고 있지만, 향후 연구는 오디오 및 비디오 데이터를 포함하는 방향으로 진행되고 있다. 이러한 모델들은 인공지능의 일반화 능력 향상에 기여하고 있으며, 여러 산업에서 실제 응용 가능성을 보여준다.

### 참고

- Klu. (n.d.). Large Multimodal Models. Retrieved from https://klu.ai/glossary/large-multimodal-models
- Shaip. (n.d.). What are Large Multimodal Models (LMMs)? Retrieved from https://www.shaip.com/blog/what-are-large-multimodal-models-lmms/
- MoveWorks. (n.d.). Multimodal Language Models. Retrieved from https://www.moveworks.com/us/en/resources/ai-terms-glossary/multimodal-language-models0
- Aimultiple. (n.d.). Large Multimodal Models. Retrieved from https://research.aimultiple.com/large-multimodal-models/
- LeewayHertz. (n.d.). Understanding Large Multimodal Models (LM

In [21]:
client.beta.assistants.delete(assistant.id)

NotFoundError: Error code: 404 - {'error': {'message': "No assistant found with id 'asst_V59x1H97uwW3nm8WdUNuwn04'.", 'type': 'invalid_request_error', 'param': None, 'code': None}}